In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import RobertaTokenizer
import torch
from tqdm import tqdm
torch.cuda.empty_cache()
import numpy as np
from sklearn.metrics import f1_score

In [6]:
df.head()

,Unnamed: 0,text,subjectivity,polarity
0,0,Love our President,OPINIONATED,POSITIVE
1,1,"Yes, same guy!!!!",NEUTRAL,NaN
2,2,Thank God we Parlerðâï¸ðºð¸.I'm grat...,OPINIONATED,POSITIVE
3,3,"This would be the best case scenario, Wednesda...",NEUTRAL,NaN
4,4,Sharpton is as non-partisan as lions are veget...,NEUTRAL,NaN


In [7]:
labels_dict = {'POSITIVE':0, 'NEGATIVE':1}
df = pd.read_csv('../data/cleaned_subj_polar__.csv', encoding='ISO-8859-1')
df = df.loc[df['subjectivity']=='OPINIONATED']
df['label'] = df['polarity'].apply(lambda x: labels_dict[x])
print(len(df))
df.head()

600


,Unnamed: 0,text,subjectivity,polarity,label
0,0,Love our President,OPINIONATED,POSITIVE,0
2,2,Thank God we Parlerðâï¸ðºð¸.I'm grat...,OPINIONATED,POSITIVE,0
6,6,The Photag has conveniently left out every ide...,OPINIONATED,NEGATIVE,1
12,12,"I admire you tremendously! Smart, quick witted...",OPINIONATED,POSITIVE,0
14,14,You hit the nsil on the head...I'm tired of he...,OPINIONATED,NEGATIVE,1


In [8]:
text = list(df.text)
labels = list(df.label)
train_texts, val_texts, train_labels, val_labels = train_test_split(text, labels, test_size=.15, stratify=labels)

In [9]:
from torch.utils.data import TensorDataset
from transformers import BertTokenizer

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [11]:
train_encode = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)

valid_encode = tokenizer.batch_encode_plus(
    val_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)
valid_input = valid_encode['input_ids']
valid_attention = valid_encode['attention_mask']
valid_labels = torch.tensor(val_labels)

input_ids = train_encode['input_ids']
attention_mask = train_encode['attention_mask']
labels = torch.tensor(train_labels)


In [12]:
train_data = TensorDataset(input_ids,
                          attention_mask,
                          labels)

valid_data = TensorDataset(valid_input,
                          valid_attention,
                          valid_labels)

In [13]:
from transformers import BertForSequenceClassification

#Finetuning pretrained BERT model with objective -> Sentence Classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data,
                              sampler = RandomSampler(train_data),
                              batch_size = BATCH_SIZE)
valid_dataloader = DataLoader(valid_data,
                              sampler = SequentialSampler(valid_data),
                              batch_size = BATCH_SIZE)


In [15]:
from transformers import AdamW, get_cosine_schedule_with_warmup

#Works better than torch.optim.Adam
optimizer = AdamW(model.parameters(),
                           lr=1e-5,
                           eps=1e-6,
                           weight_decay=1e-1)
#chosen after setting = 3 (f1 -> 0.71), 5 (f1 -> 0.77) but acpc for many classes is 0
#10 Epochs helps with generalization
EPOCHS = 10 

#Works better than torch.optim.CosineAnnealingLR and transformers.get_linear_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [16]:
def f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [17]:
#Per Class Accuracy
def acpc(preds, labels):
    code_dict = {val:key for key,val in labels_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for idx in np.unique(labels_flat):
        y_hat = preds_flat[labels_flat==idx]
        y = labels_flat[labels_flat==idx]
        print(f'Class: {code_dict[idx]}')
        print(f'Accuracy: {len(y_hat[y_hat==idx])/ len(y)}\n')

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [19]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in valid_dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [20]:
vloss = []
score = []
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()

    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids' : batch[0],
                  'attention_mask': batch[1],
                  'labels' : batch[2]
                  }
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'./bert polarity/BERT_polarity_epoch_{epoch}.pt')

    tqdm.write(f'\nEpoch {epoch}')

    avg_training_loss = total_loss/len(train_dataloader)            
    tqdm.write(f'Training loss: {avg_training_loss}')

    val_loss, predictions, actual = evaluate(valid_dataloader)
    score_f1 = f1(predictions, actual)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {score_f1}')
    vloss.append(val_loss)
    score.append(score_f1)

  0%|                                                                                           | 0/10 [00:35<?, ?it/s]


Epoch 1
Training loss: 0.495698748389259


Epoch 2:   0%|                                                                                  | 0/64 [00:00<?, ?it/s]

Validation loss: 0.15310885570943356
F1 Score (Weighted): 1.0



 10%|████████▎                                                                          | 1/10 [01:13<05:29, 36.59s/it]

...

Validation loss: 0.0026867854758165777
F1 Score (Weighted): 1.0



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [06:12<00:37, 37.09s/it]


Epoch 10
Training loss: 0.0032667596206010785


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:13<00:00, 37.39s/it]

Validation loss: 0.0026765443617478013
F1 Score (Weighted): 1.0


In [21]:
for i, pair in enumerate(zip(vloss, score)):
    print(i, pair[0], pair[1])

0 0.15310885570943356 1.0
1 0.022375446899483602 1.0
2 0.00871077529154718 1.0
3 0.004871172791657348 1.0
4 0.06245090771699324 0.9777667984189722
5 0.0033734951672765114 1.0
6 0.002955494448542595 1.0
7 0.002763028761061529 1.0
8 0.0026867854758165777 1.0
9 0.0026765443617478013 1.0


In [23]:
model.to(device)
model.load_state_dict(torch.load('./bert polarity/BERT_polarity_epoch_10.pt', map_location=torch.device('cpu')))

_, predictions, actual = evaluate(valid_dataloader)
print(acpc(predictions, actual))
labs = {v:k for k,v in labels_dict.items()}
for i in range (len(predictions)):
    print(labs[np.argmax(predictions[i])], val_texts[i])
    print()

Class: POSITIVE
Accuracy: 1.0

Class: NEGATIVE
Accuracy: 1.0

None
NEGATIVE She sucks. Was never qualified. Loser

POSITIVE Rock on brother this says it all.. love it

NEGATIVE Ironic the very media platform that has gone to great lengths to incite devise ness is projecting a victim of their bullshit is being silenced. This is a bad movie

POSITIVE It's very deep and very real!

POSITIVE Absolutely great first move by Trump and the GOP!!

NEGATIVE a href="/profile/HughHewitt/posts" class="at">@HughHewitt</a> You maybe a lot of things but never knew deaf dumb and blind were part of your weaknesses? I am exactly your age and remember the past as u put it. What if 75 million Trumpers stop voting? Clean what up in 4 years? Wonât be anything to clean up! Iâm done voting and have all my life! Done! And done listening to pundits! U are lost!

NEGATIVE Sickening

POSITIVE Good Morning! Have a great day!

POSITIVE Absolutely.

NEGATIVE Clearly not a secure election once again.

POSITIVE May